# 数据探索 处理用户行为数据

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import *
%matplotlib inline

### 1.读入数据

In [ ]:
# File = '../action02/000000_0'

# #colInfo =['user_id','sku_id','time','model_id','type','cate','brand']
# colInfo =['user_id','sku_id','cate','brand', \
#           'sum_type1','sum_type2','sum_type3','sum_type4','sum_type5','sum_type6']

# data = pd.read_table(File,sep='\001',names=colInfo)
# data.head() #显示数据的前5行

### 2.评论数据探索

In [ ]:
df.describe()

In [ ]:
# 查看数据条数
len(df)

### 3.数据清洗

### 4.查看属性的取值分布情况 

In [ ]:
len(df['dt'].unique())

In [ ]:
# 查看获取评论数据的时间窗口
df['dt'].unique()

In [ ]:
# 查看每一个截止日期的数据条数
[len(df[df['dt'] == x]) for x in df['dt'].unique()]

In [ ]:
#查看每一个时间窗口的商品id的个数
[len(df[df['dt'] == x]['sku_id'].unique()) for x in df['dt'].unique()]

In [ ]:
len(df['sku_id'].unique())

In [ ]:
# 根据数据探索的结果推测,总共有46546中商品,每一个商品,在12个时间窗口,分别采集了评论数据
# 因此可以把评论数据看成是时间序列的数据,在每一个窗口有3个维度

In [ ]:
#查看每一个属性取值的数据条数
df['comment_num'].value_counts()

In [ ]:
# 做离散化处理
df['comment_num_0'] = 0.0
df['comment_num_1'] = 0.0
df['comment_num_2'] = 0.0
df['comment_num_3'] = 0.0
df['comment_num_4'] = 0.0
# 离散化赋值
df.loc[df['comment_num'] == 1,'comment_num_1']=1
df.loc[df['comment_num'] == 2,'comment_num_2']=1
df.loc[df['comment_num'] == 3,'comment_num_3']=1
df.loc[df['comment_num'] == 4,'comment_num_4']=1
df.loc[df['comment_num'] == 0,'comment_num_0']=1
# 查看结果
df.head()

### 5.特征处理 

In [ ]:
# 处理好的数据写入到文件
# 丢弃处理之前的属性列
#colDrop = ['dt','comment_num']
colDrop = ['comment_num']
df_sel =df.drop(colDrop,axis=1).copy()

In [ ]:
df_sel.head()

In [ ]:
# 对经过列筛选的数据做数据归并
# 对has_bad_comment 做合计,求发生差评的时间窗口次数
# 对bad_comment_rate 求均值和方差
# 对comment_num_0 ~ comment_num_4 做合计,求出相同取值的时间窗口次数

In [ ]:
grouped_sum=df_sel[['comment_num_0','comment_num_1','comment_num_2','comment_num_3','comment_num_4']].groupby(df_sel['sku_id']).sum()

In [ ]:
grouped_sum

In [ ]:
# 差评率的均值
grouped_mean=df_sel['bad_comment_rate'].groupby(df_sel['sku_id']).mean()
grouped_mean

In [ ]:
# 差评率的方差
grouped_var=df_sel['bad_comment_rate'].groupby(df_sel['sku_id']).var()
grouped_var

In [ ]:
# 创建输出用的df
df_out = grouped_sum.copy()
df_out['bad_comment_rate_mean'] = grouped_mean
df_out['bad_comment_rate_var'] = grouped_var
df_out['has_bad_comment_times'] = df_sel['has_bad_comment'].groupby(df_sel['sku_id']).sum()

In [ ]:
df_out.head()

In [ ]:
# 查看某一个特征的取值分布情况
df_sel['has_bad_comment'].groupby(df_sel['sku_id']).sum().unique()

In [ ]:
df_out.to_csv('../comment_df_sel.csv',index=True)